In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import pyhocon
import torch
import joblib
import numpy as np
from RobotCommandClassifier.MyMultilabel import (MyMultiLabelClassificationModel,
                                                 MyMultiLabelClassificationArgs,
                                                 MyBertForMultiLabelSequenceClassification)
from transformers import (
    BertConfig,
    BertTokenizer
)

In [2]:
configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/CustomML.conf")
CONFIG = configFileContent['MyMultiTiny2_fold1'].as_plain_ordered_dict()

In [3]:
model_args = MyMultiLabelClassificationArgs(**CONFIG["Model"]["Args"])
num_labels = sum(CONFIG["Model"]['num_sublabels_per_biglabel'])
bertconfig = BertConfig.from_pretrained("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/MyMultiTiny2/fold_1/models/checkpoint-63780-epoch-10",
                                       num_labels=num_labels, **model_args.config)
tokenizer = BertTokenizer.from_pretrained("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/MyMultiTiny2/fold_1/models/checkpoint-63780-epoch-10",
                                         do_lower_case=model_args.do_lower_case)

In [5]:
model = MyBertForMultiLabelSequenceClassification.from_pretrained("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/MyMultiTiny2/fold_1/models/checkpoint-63780-epoch-10",
                                                                  bertconfig, 
                                                                  num_sublabels_per_biglabel=CONFIG["Model"]["num_sublabels_per_biglabel"],
                                                                  add_attention_for_labels=CONFIG["Model"]["add_attention_for_labels"])

In [6]:
model.to(torch.device("cuda"))

MyBertForMultiLabelSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1

In [7]:
phrases = ["а нука, робот, подь сюды", "иди к тому дереву, которое за домом"]
model_inputs = tokenizer.batch_encode_plus(
                phrases, return_tensors="pt", padding=True, truncation=True
            )

{'input_ids': tensor([[    2,   312, 50744,   603,    16, 51960,    16,  1464,  1090, 58079,
           1758,     3],
         [    2, 53967,   322,  3567, 59282,    16,  8284,   650, 37245,     3,
              0,     0]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]], device='cuda:0')}

In [9]:
enc = joblib.load(CONFIG["output_dir"]+"/onehotenc.joblib")

In [18]:
def predict_for_prases(phrases):
    with torch.no_grad():
        model_inputs = tokenizer.batch_encode_plus(
                    phrases, return_tensors="pt", padding=True, truncation=True
                )
        model_inputs = {key: value.to(torch.device("cuda")) for key, value in model_inputs.items()}
        logits = model(**model_inputs)
        logits = logits[0].cpu()
        predictions_2 = np.zeros((len(logits), len(enc.categories_)))
        for i in range(predictions_2.shape[0]):
            shift = 0
            for j in range(len(enc.categories_)):
                if "OneHotArgs" in CONFIG["Data"] and CONFIG["Data"]["OneHotArgs"].get('drop', None) == "first":
                    label_logits = logits[i, shift:shift+len(enc.categories_[j])-1]
                    if sum(label_logits>=0.5)==0:
                        predictions_2[i,j] = 0
                    else:
                        predictions_2[i,j] = np.argmax(label_logits)+1
                    shift += len(enc.categories_[j])-1
                else:
                    predictions_2[i,j] = np.argmax(logits[i, shift:shift+len(enc.categories_[j])])
                    shift += len(enc.categories_[j])
            assert shift==logits.shape[1]
    return predictions_2

# test

In [11]:
from RobotCommandClassifier import utils
from RobotCommandClassifier import *

from sklearn.preprocessing import OneHotEncoder
train_x_df, train_y_df, valid_x_df, valid_y_df, test_x_df, test_y_df = utils.load_data(**CONFIG["Data"])

if CONFIG["Data"].get("add_y_to_x", False):
    with open(CONFIG["Data"]["y_descriptions_path"], "r") as f:
        y_descriptioons = json.load(f)
    train_x_df = train_x_df["y"].map(lambda y: y_descriptioons[int(y)]) + ": " + train_x_df["x"]
    valid_x_df = valid_x_df["y"].map(lambda y: y_descriptioons[int(y)]) + ": " + valid_x_df["x"]
    test_x_df = test_x_df["y"].map(lambda y: y_descriptioons[int(y)]) + ": " + test_x_df["x"]

if CONFIG["Data"].get("predict_label_flag", False):
    if "y" in CONFIG["Data"]["target_columns"]:
        raise ValueError("Указан флаг для использования только бинарных лейблов. Предполагается, что в таком случае 'y' не должен ыть среди target_columns.")
    train_y_df[train_y_df!=0] = 1
    valid_y_df[valid_y_df!=0] = 1
    test_y_df[test_y_df!=0] = 1
if CONFIG["Type"] in ["simple_ml_multilabel_classifier", "mymulti_classifier"]:
    if CONFIG["Data"].get("predict_label_flag", False):
        labels = train_y_df.values.tolist()
    else:
        enc = OneHotEncoder(**CONFIG["Data"].get("OneHotArgs", {}))
        enc.fit(train_y_df.values)

        labels = []
        encoded_labels = enc.transform(train_y_df.values).toarray().astype(int)
        for i in range(train_y_df.shape[0]):
            labels.append(encoded_labels[i].tolist())
    train_df = pd.DataFrame(list(zip(train_x_df, labels)))        
else:
    train_df = pd.concat([train_x_df, train_y_df], axis=1)
train_df.columns = ["text", "labels"]

if CONFIG["Type"] == "simple_ml_classifier":
    num_labels = len(train_y_df.iloc[:,0].unique())
else:
    num_labels = len(labels[0])

In [19]:
predictions = predict_for_prases(test_x_df.values.tolist())

In [23]:
predictions

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [2., 1., 0., ..., 0., 0., 0.],
       ...,
       [6., 0., 0., ..., 3., 0., 0.],
       [6., 0., 0., ..., 3., 0., 0.],
       [6., 0., 0., ..., 3., 0., 0.]])

In [31]:
predictions_2 = []
for line in test_x_df.values.tolist():
    pred = predict_for_prases([line])
    predictions_2.append(pred)
predictions_2 = np.array(predictions_2).squeeze()

In [21]:
#result = utils.calculate_metrics_2(test_y_df.iloc[:,1:], predictions_2[:,1:], display=True) # исключить Y из оценки
result = utils.calculate_metrics_2(test_y_df, predictions, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_y_df, predictions, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       1.00      0.50      0.67         6
         1.0       0.60      0.75      0.67         4
         2.0       0.93      0.93      0.93        15
         3.0       1.00      1.00      1.00         3
         6.0       1.00      1.00      1.00       236
         8.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0

    accuracy                           0.98       264
   macro avg       0.65      0.60      0.61       264
weighted avg       0.99      0.98      0.98       264

direction
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       246
           1       0.82      0.75      0.78        12
           2       0.67      0.67      0.67         3
           4       1.00      1.00      1.00         3

    accuracy                           0.98       264
   macro avg       0.87      0.85      0.86       264
weight

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

92.0 	 correct_samples_perc
98.0 	 [action]_acc
61.0 	 [action]_macrof1
98.0 	 [direction]_acc
86.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
100.0 	 [degshours]_macrof1
98.0 	 [object1]_acc
98.0 	 [object1]_macrof1
100.0 	 [nearest]_acc
100.0 	 [nearest]_macrof1
100.0 	 [relation1]_acc
75.0 	 [relation1]_macrof1
98.0 	 [object2]_acc
98.0 	 [object2]_macrof1
99.0 	 [relation2]_acc
75.0 	 [relation2]_macrof1
99.0 	 [object3]_acc
99.0 	 [object3]_macrof1
100.0 	 [self]_acc
100.0 	 [self]_macrof1
99.0 	 [gaze]_acc
50.0 	 [gaze]_macrof1
87.0 	 avg_macro_f1
99.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [38]:
#result = utils.calculate_metrics_2(test_y_df.iloc[:,1:], predictions_2[:,1:], display=True) # исключить Y из оценки
result = utils.calculate_metrics_2(test_y_df, predictions_2, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_y_df, predictions_2, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       1.00      0.50      0.67         6
         1.0       0.60      0.75      0.67         4
         2.0       0.93      0.93      0.93        15
         3.0       1.00      1.00      1.00         3
         6.0       1.00      1.00      1.00       236
         8.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0

    accuracy                           0.98       264
   macro avg       0.65      0.60      0.61       264
weighted avg       0.99      0.98      0.98       264

direction
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       246
           1       0.82      0.75      0.78        12
           2       0.67      0.67      0.67         3
           4       1.00      1.00      1.00         3

    accuracy                           0.98       264
   macro avg       0.87      0.85      0.86       264
weight

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

92.0 	 correct_samples_perc
98.0 	 [action]_acc
61.0 	 [action]_macrof1
98.0 	 [direction]_acc
86.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
100.0 	 [degshours]_macrof1
98.0 	 [object1]_acc
98.0 	 [object1]_macrof1
100.0 	 [nearest]_acc
100.0 	 [nearest]_macrof1
100.0 	 [relation1]_acc
75.0 	 [relation1]_macrof1
98.0 	 [object2]_acc
98.0 	 [object2]_macrof1
99.0 	 [relation2]_acc
75.0 	 [relation2]_macrof1
99.0 	 [object3]_acc
99.0 	 [object3]_macrof1
100.0 	 [self]_acc
100.0 	 [self]_macrof1
99.0 	 [gaze]_acc
50.0 	 [gaze]_macrof1
87.0 	 avg_macro_f1
99.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu